Créé le 9 février 2021

**Compétition Kaggle** 

**Team : Les Kagglettes**
 
Auteurs : KIRED Nour & ROSSI Valentine

Promo : M1 SID

# Récupération des données

## Lien entre Colab et le Drive

In [ ]:
# lien avec Colab
%load_ext google.colab.data_table
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# création de la date
import os.path, time
print("created: %s" % time.ctime(os.path.getctime("/content/drive/MyDrive/kaggle Project/Données/comments_students.csv"))) 

created: Tue Feb  9 18:43:35 2021


## Importation des librairies

In [ ]:
# installations de librairies
!pip install transformers

     |████████████████████████████████| 2.2MB 12.6MB/s 
     |████████████████████████████████| 870kB 36.7MB/s 
     |████████████████████████████████| 3.3MB 41.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=f081a8093abbeab3e8b51a171e9e25e3e83bb6c86d68e50e1c9426d31c34eb76
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import random
# Modele lgbm
import lightgbm as lgb
import sklearn

# Reseau
from pprint import pprint
import networkx as nx
from textblob import Word
from textblob import TextBlob

In [ ]:
# PARTIE TRAITEMENT DE TEXTE

# Maths
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


#  traitement de texte
from wordcloud import WordCloud,STOPWORDS
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords=stopwords.words('english')

# Traitement du langage et recherche d'informations
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Modeles de prediction 
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import BernoulliNB

from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import make_regression

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Pas de message d'alerte
import warnings
warnings.filterwarnings('ignore')
### barre de chargement
from tqdm import tqdm

## Importation des données

In [ ]:
## importation des chemins et données utiles
feature_dossier = "/content/drive/MyDrive/kaggle Project/Données/Data_Feature/"
comments_students = "/content/drive/MyDrive/kaggle Project/Données/comments_students.csv"
tests="/content/drive/MyDrive/kaggle Project/tests"
all_feature=feature_dossier + "data_all_feature.csv"
df_fin = pd.read_csv(comments_students)[['ups', 'id']]

# **PARTIE 1**: Exploration et traitement des données




## Description et affichage des données

In [ ]:
## lecture du fichier csv
data=pd.read_csv(comments_students)

In [ ]:
# Lecture de 5 lignes de manière aléatoire
data.sample(5)

,created_utc,ups,subreddit_id,link_id,name,subreddit,id,author,body,parent_id
1685272,1431534668,2.0,t5_2qh1i,t3_35u9yx,t1_cr7vm6i,AskReddit,cr7vm6i,crazu,I still use my parents old TomTom from 2004. I...,t1_cr7vbx2
2256604,1431891616,3.0,t5_2qh1i,t3_36959m,t1_crc700d,AskReddit,crc700d,Ragnar_The_Dane,The purpose of a justice system is first and f...,t1_crc6ugp
4112155,1433034572,NaN,t5_2qh1i,t3_37wocp,t1_crqefop,AskReddit,crqefop,Y_dilligaf,"I'm talking, post birth 2 fully developed huma...",t1_crqeb3o
3156424,1432467745,2.0,t5_2qh1i,t3_372xmv,t1_crj8vvo,AskReddit,crj8vvo,MyHeadIsNotRight,God,t3_372xmv
552494,1430831729,2.0,t5_2qh1i,t3_34t9e9,t1_cqz16lo,AskReddit,cqz16lo,honeybadgergrrl,"Yeah, she probably brought home more money tha...",t1_cqyt8kr


##### partie exploration des données

In [ ]:
# Affichage du type des colonnes
data.dtypes 
# Supprission de data pour ne pas consommer bcp d'espace de Ram
del data

**Description des colonnes :**
- created_utc : heure de création (format heure UTC seconde)
- ups : score du commentaire, c'est l'attribut à prédire.
- subreddit_id : identifiant du subreddit dans lequel se trouve l'objet
- link_id : identifiant du lien dans lequel se trouve le commentaire
- name : nom complet du commentaire
- subreddit : subreddit de chose excluant le / r / pre x. \photos"
- id : identifiant de l'article
- author : nom du compte de l'affiche
- body : texte brut (texte non formaté qui inclut les caractères de balisage brut tels que ** pour gras)
- parent_id : identifiant de l'élément auquel le commentaire est une réponse, qu'il s'agisse du lien ou du commentaire qu'il contient


## Creation des features

#### 1er feature (closeness)

In [ ]:
# Chargement des données et suppression de colonnes inutiles
df = pd.read_csv(comments_students).drop(columns=['subreddit_id', 'subreddit', 'id'])

In [ ]:
# Création du graph
g = nx.DiGraph()
g.add_nodes_from(df.link_id, type="link")
g.add_nodes_from(df.name, type="comment")
g.add_edges_from(df[["name", "parent_id"]].values, link_type="parent")

In [ ]:
# Features correspondantes
close_cent = nx.degree_centrality(g)
closeness = pd.DataFrame()
closeness['close_cent'] = [close_cent[x] for x in df["name"]]


In [ ]:
# Features correspondantes
close_cent = nx.degree_centrality(g)
closeness = pd.DataFrame()
closeness['close_cent'] = [close_cent[x] for x in df["name"]]
closeness.to_csv(feature_dossier + "closeness.csv")

del g, close_cent, closeness,df

#### 2e feature (closeness_author)

In [ ]:
# Chargement des données avec les colonnes utiles
df = pd.read_csv(comments_students)[['name', 'author', 'parent_id', 'link_id']]

In [ ]:
# Features correspondantes
dic_aut_name : dict = {df['name'].iloc[i]: df['author'].iloc[i] for i in range(len(df))}

In [ ]:
liste : list = []
for i in tqdm(df['parent_id'].values):
    if i in dic_aut_name.keys():
        liste.append(dic_aut_name[i])
    else:
        liste.append(i)
df['new_parent_id'] : list= liste

100%|██████████| 4234970/4234970 [00:04<00:00, 1021800.89it/s]


In [ ]:
# Création du graph
g_di = nx.DiGraph()
g_di.add_nodes_from(df.link_id, type="link")
g_di.add_nodes_from(df.author, type="comment")
g_di.add_edges_from(df[["author", "new_parent_id"]].values)

close_cent = nx.degree_centrality(g_di)

df['close_cent_aut'] = [close_cent[i] for i in tqdm(df['author'].values)]
df = pd.DataFrame(df['close_cent_aut'])

100%|██████████| 4234970/4234970 [00:02<00:00, 1585336.45it/s]


In [ ]:
# Exportation des features
df.to_csv(feature_dossier + "closeness_author.csv")
del df, dic_aut_name, liste, i, g_di, close_cent

#### 3e feature ( betweenness_centrality_author)

In [ ]:
# Chargement des données avec les colonnes utiles
df = pd.read_csv(comments_students)[['name', 'author', 'parent_id', 'link_id']]

In [ ]:
# Features correspondantes
dic_aut_name : dict = {df['name'].iloc[i]: df['author'].iloc[i]
                for i in range(len(df))}

In [ ]:
liste : list = []
for i in tqdm(df['parent_id'].values):
    if i in dic_aut_name.keys():
        liste.append(dic_aut_name[i])
    else:
        liste.append(i)
df['new_parent_id'] : list = liste

100%|██████████| 4234970/4234970 [00:04<00:00, 1026750.67it/s]


In [ ]:
# Création du graph
g_di = nx.DiGraph()
g_di.add_nodes_from(df.link_id, type="link")
g_di.add_nodes_from(df.author, type="comment")
g_di.add_edges_from(df[["author", "new_parent_id"]].values)

bet_cent = nx.betweenness_centrality(g_di, k=100)
bet_cent_aut = pd.DataFrame([bet_cent[i] for i in tqdm(
    df['author'].values)], columns=['bet_cent_aut'])
bet_cent_aut = bet_cent_aut['bet_cent_aut']

In [ ]:
# Exportation des features
bet_cent_aut.to_csv(feature_dossier + "betweenness_centrality_author.csv")
del df, dic_aut_name, liste, i, g_di, bet_cent_aut

#### 4e feature(comment_depth) ( trop long )

feature pensée, mais malheuresement on a pas pu l'executer car trop long.

In [ ]:
def it_prof(dic_name_depth, prof, list_noeud):
    for tt in list_noeud:
        if prof == 0 and tt[1] == '1':
            prof += 2
        dic_name_depth.update({tt: prof})
        it_prof(dic_name_depth, prof+1, list(g.predecessors(tt)))

In [ ]:
df = pd.read_csv(comments_students)[['link_id', 'name', 'parent_id']]
liste_unique_link = np.unique(df['link_id'])
dic_name_depth : dict = {}

In [ ]:
for link in tqdm(liste_unique_link):
    df2 = df[df['link_id'] == link]
    g = nx.DiGraph()
    g.add_nodes_from(df2.link_id, type="link")
    g.add_nodes_from(df2.name, type="comment")
    g.add_edges_from(df2[["name", "parent_id"]].values)
    list_noeud = [node for node in list(
        g.nodes()) if list(g.neighbors(node)) == []]
    prof : int = 0
    it_prof(dic_name_depth, prof, list_noeud)

In [ ]:
df['com_depth'] = [dic_name_depth[i] for i in df['name'].values]
df[['com_depth']].to_csv(feature_dossier + 'comment_depth.csv')

del df, g, liste_unique_link, dic_name_depth, df2, link, prof, list_noeud, it_prof

#### 5e Feature (creaction des topics)

##### Fonctions

In [ ]:
def cleanedWords(raw_sentence):
  """Documentation
    Parameters:
        raw_sentence: un commentaire (une ligne qui correspond à un commentaire du fichier csv)
    Out:
        le commentaire nettoyé en supprimant les chiffres les caracteres speciaux ainsi qu'en le mettant en miniscule
    """
    return (re.sub("[^a-zA-Z]", " ", raw_sentence.lower()))

def remove_stopwords(texts):
    """"Documentation
      Parameters:
            textes : les bodies
      Out :
            les bodies sans mots vides

    """
    return [word for word in simple_preprocess(texts) if word not in stopWords] 

def sent_to_words(sentences):
    words = []
    for sentence in tqdm(sentences):
        words.append(str(sentence).split())  
    return words
        
def lemmatization(texts):

    texts_out = []
    for sent in tqdm(texts):
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])
    return texts_out


##### Nettoyage

In [ ]:
col_body : list = []
chunksize : int  = 10 ** 5


for i, chunk in enumerate(pd.read_csv("/content/drive/MyDrive/M1 SID/kaggle Project/Données/comments_students.csv", chunksize=chunksize)):
    chunk = chunk.astype({'body': str})
    print("chunk ", i, " begin")
    data_words = [cleanedWords(sentence) for sentence in tqdm(chunk['body'].values)]
    print("chunk ", i, "data_words ok")

    # Remove Stop Words
    data_words_nostops = []
    for doc in tqdm(data_words):
      result=remove_stopwords(doc)
      if result!=[]:
        data_words_nostops.append(result)
      else :
        data_words_nostops.append([doc])

    print("chunk ", i, "data_words_nostops ok")

    # Do lemmatization keeping 
    data_lemmatized = lemmatization(data_words_nostops)

    print("chunk ", i, "data_lemmatized ok")

    col_body += data_lemmatized
    print("chunk ", i, " ok")


del i, chunk, chunksize, data_words, data_words_nostops, data_lemmatized


 12%|█▏        | 12483/100000 [00:00<00:00, 124827.44it/s]

chunk  0  begin



100%|██████████| 100000/100000 [00:00<00:00, 110044.78it/s]

  1%|▏         | 1280/100000 [00:00<00:07, 12748.80it/s]

chunk  0 data_words ok



100%|██████████| 100000/100000 [00:10<00:00, 9941.18it/s]

  0%|          | 20/100000 [00:00<08:32, 195.05it/s]

chunk  0 data_words_nostops ok



 12%|█▏        | 12427/100000 [00:38<04:38, 313.93it/s]

KeyboardInterrupt: ignored

##### Recuperation du CSV propre

In [ ]:
### enrigetrement du body nottoyé dans un fichier csv
df = pd.DataFrame()
df['body'] = col
df.to_csv("/content/drive/MyDrive/kaggle Project/Données/clean_body.csv")

In [ ]:
# recharement du col_body depuis clean_body.csv 
col_body = pd.read_csv("/content/drive/MyDrive/kaggle Project/Données/Data_Feature/clean_body.csv")

In [ ]:
### nettoyage du col body afin de l'exmploiter
for j in tqdm(range(len(col_body))):
    for i in ['[', ']', "'", " "]:
        col_body[j] = col_body[j].replace(i, "")
    col_body[j] = col_body[j].split(",")

col_body = col_body.to_list()

del j, i

##### creation des topics

In [ ]:
# Creation d'un dictionnaire de tous les mots du col body ()
id2word = corpora.Dictionary(col_body)
print("dict ok")

# creaction du corpus (texts) 
texts = col_body
print("text ok")

# vectorisation de nos mots 
corpus = [id2word.doc2bow(text) for text in texts]
print("corpus ok")

del col_body, texts

In [ ]:
limit = 20
start = 2
step = 2

for num_topics in tqdm(range(start, limit, step)):
    lda_model = gensim.models.LdaModel(corpus,
                                       num_topics=num_topics,
                                       id2word=id2word,
                                       random_state=100,
                                       alpha='auto')

    coherencemodel = CoherenceModel(
        model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    print("coherence avec ", num_topics, " topics : ",
          coherencemodel.get_coherence())


del limit, start, step, coherencemodel

on prendra 12 topics

In [ ]:
## entrainement d'un modele qui va nous permettre de clusteriser nos 
## commentaires par topics
lda_model = gensim.models.LdaModel(corpus,
                                   num_topics=12,
                                   id2word=id2word,
                                   random_state=100,
                                   alpha='auto')

del id2word

In [ ]:
def format_topics_sentences(ldamodel, corpus):
    """"Documentation
      Parameters:
            ldamodel : notre modele entrainé pour clusteriser nos commentaire par topic
            corpus : tous nos commentaires
      Out :
            chaque commentaire est associé à son topic (creation d'une column dans notre df)

    """
    # Iinitialisation de notre output df
    sent_topics_df = pd.DataFrame()

    # recupere le topic qui maximise le score pour chaque commentaire
    nb_comm = len(corpus)
    list_topic = [int(sorted(lda_model[corpus][i], key=lambda x: (
        x[1]), reverse=True)[0][0]) for i in tqdm(range(nb_comm))]
    sent_topics_df['Dominant_Topic'] = list_topic

    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(
    ldamodel=lda_model, corpus=corpus)
del lda_model

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()

del df_topic_sents_keywords

df_dominant_topic.columns = ['Document_No', 'Dominant_Topic']

print("topic / comm ok")

In [ ]:
df = pd.DataFrame()
df['Topic'] = df_dominant_topic['Dominant_Topic']
## enrigestrement de notre premiere feature dans un dossier csv
df.to_csv(feature_dossier + "topic.csv")

del df_dominant_topic, df

#### 6e feature ( sentiments)

In [ ]:
col_polarity : list = []
col_subjectivity : list= []
chunksize : int = 10 ** 5

for i, chunk in enumerate(pd.read_csv(comments_students, chunksize=chunksize)):
    print("chunk ", i, " begin")
    for j in chunk['body']:
        polarity, subjectivity = TextBlob(str(j)).sentiment

        col_polarity.append(polarity)
        col_subjectivity.append(subjectivity)
    print("chunk ", i, " ok \n")

del i, chunk, chunksize, polarity, subjectivity

df = pd.DataFrame()
df['polarity'] = col_polarity
df['subjectivity'] = col_subjectivity
df.to_csv(feature_dossier + "sentiment_feature.csv")

del df, col_polarity, col_subjectivity

#### 7e feature (nb de mots body)

In [ ]:
df = pd.DataFrame(pd.read_csv(comments_students)['body'])
df = pd.concat([df, pd.DataFrame([len(str(x).split(' '))
                                  for x in df['body']], columns=['nb_mot_body'])], axis=1)
df = df.drop('body', axis=1)

df.to_csv(feature_dossier + "nb_mot_body.csv")
del df

#### 8e feature(post_comm)

In [ ]:
# Chargement du fichier CSV
df = pd.read_csv(comments_students)[['name', 'link_id']]

# Ajout du post ( <=> t3) correspondant au commentaire
link_new_id = {df['link_id'][i]: i for i in tqdm(range(len(df)))}
df['link_new_id'] = [link_new_id[i] for i in df['link_id'].values]
df = pd.DataFrame(df['link_new_id'])

# Exportation de features
df.to_csv(feature_dossier + "post_comm.csv")
del df, link_new_id

#### 9e feature (pop_post)

In [ ]:
# Chargement du fichier CSV
df = pd.read_csv(comments_students)[['link_id']]

# Ajout de la popularité du post ( <=> nombre de commetaires dans le post = t3)
df['pop'] = [1 for i in range(len(df))]
df['Unnamed: 0'] = df.index
df = pd.DataFrame(df.merge(df[['link_id', 'pop']].groupby(['link_id']).sum(), on='link_id', suffixes=(
    '', '_post'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0']).drop(columns=['pop'])['pop_post'])

df = pd.DataFrame(df['pop_post'])

# Exportation de features
df.to_csv(feature_dossier + "pop_post.csv")

del df

#### 10e feature (time )

In [ ]:
# Chargement du fichier CSV
df = pd.read_csv(comments_students)[
    ['created_utc', 'link_id', 'author', 'name', 'parent_id']]

# Ajout du temps de réponse depuis la création du post ( <=> 1er commentaire sur le post)
df['Unnamed: 0'] = df.index
df = df.merge(df[['link_id', 'created_utc']].groupby('link_id').min(), on='link_id', suffixes=(
    '', '_min'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0'])

df['time_1st_comment_post'] = df['created_utc']-df['created_utc_min']
df = df.drop(columns=['created_utc_min'])

# Ajout du temps de réponse depuis 1er post de l'auteur
df['Unnamed: 0'] = df.index
df = df.merge(df[['author', 'created_utc']].groupby('author').min(), on='author', suffixes=(
    '', '_aut'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0'])

df['time_1st_comment_author'] = df['created_utc']-df['created_utc_aut']

# Ajout du temps de réponse au commentaire père
df['Unnamed: 0'] = df.index
df = df.merge(df[['name', 'created_utc']], left_on='parent_id', right_on='name', suffixes=(
    '', '_parent'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0'])

df['time_answer'] = (df['created_utc']-df['created_utc_parent']).fillna(0)

# On ne garde que les colonnes des features
df = df[['time_1st_comment_post', 'time_1st_comment_author', 'time_answer']]

# Exportation de features
df.to_csv(feature_dossier + "time_feature.csv")

del df

#### 11e feature (nb comm par auteur)

In [ ]:
# Chargement du fichier CSV
df = pd.read_csv(comments_students)[['author']]

# Ajout de du nombre de commentaires par autheur
df['Unnamed: 0'] = df.index
df['nb_com'] = [1 for i in range(len(df))]
df = df.merge(df[['author', 'nb_com']].groupby('author').count(), on='author', suffixes=(
    '', '_aut'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0'])

df = pd.DataFrame(df['nb_com_aut'])

# Exportation de features
df.to_csv(feature_dossier + "nb_comm_per_author.csv")
del df

#### 12e- 13e- 14e features (nombreDescendants, nombreEnfants, ratioEnfantsDescendants)

In [ ]:
df=pd.read_csv(comments_students)
G = nx.DiGraph()

G.add_nodes_from(list(df['name']))


for i in tqdm(range(df.shape[0])) :
    
    G.add_edge(df['parent_id'].iloc[i],df['name'].iloc[i])
nodesPeres = list(df['parent_id'])
nodesFils = list(df['name'])
nombreEnfants = [len(list(G.successors(i))) for i in (df['name'])]
nombreDescendants = [len(list(nx.nodes(nx.dfs_tree(G, i)))) for i in ( df['name']) ]
df['nombreEnfants'] = nombreEnfants
df['nombreDescendants'] = nombreDescendants
df['ratioEnfantsDescendants'] = df['nombreEnfants']/df['nombreDescendants']

100%|██████████| 4234970/4234970 [02:06<00:00, 33528.43it/s]


In [ ]:
# Exportation de features
df.to_csv(feature_dossier + "nombreDescendants.csv")
df.to_csv(feature_dossier + "nombreEnfants.csv")
df.to_csv(feature_dossier + "ratioEnfantsDescendants.csv")

#### 15e feature (mean_mod_author)

In [ ]:
# Chargement et séparation des comments
df = pd.read_csv(comments_students)
train = df[~np.isnan(df['ups'])][['ups', 'author', 'parent_id']]
test = df[np.isnan(df['ups'])][['ups', 'author', 'parent_id']]
del df

# ---------------------------------------------------------------------------------------------------
# Partie train
# Ajout moyenne ups author
train['Unnamed: 0'] = train.index
train = pd.DataFrame(train.merge(train[['ups', 'author']].groupby(['author']).mean(
), on='author', suffixes=('', '_mean_author')).sort_values(['Unnamed: 0']).set_index(['Unnamed: 0']))


# Ajout mode ups author
train['Unnamed: 0'] = train.index
temp = train[['author', 'ups']].groupby(['author'])['ups'].agg(pd.Series.mode)
for i in temp.index:
    if isinstance(temp[i], np.ndarray):
        temp[i] = temp[i].mean()

train = pd.DataFrame(train.merge(pd.DataFrame(temp), on='author', suffixes=(
    '', '_mode_author'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0']))
del temp


# Ajout moyenne ups des réponse au commentaire père
train['Unnamed: 0'] = train.index
train = pd.DataFrame(train.merge(train[['ups', 'parent_id']].groupby(['parent_id']).mean(
), on='parent_id', suffixes=('', '_mean_ans_parent')).sort_values(['Unnamed: 0']).set_index(['Unnamed: 0']))


# Ajout mode ups des réponse au commentaire père
train['Unnamed: 0'] = train.index
temp = train[['ups', 'parent_id']].groupby(
    ['parent_id'])['ups'].agg(pd.Series.mode)
for i in temp.index:
    if isinstance(temp[i], np.ndarray):
        temp[i] = temp[i].mean()

train = pd.DataFrame(train.merge(pd.DataFrame(temp), on='parent_id', suffixes=(
    '', '_mode_ans_parent'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0']))
del temp


# ---------------------------------------------------------------------------------------------------
# Partie test
# Ajout moyenne ups author
test['Unnamed: 0'] = test.index
test = test.merge(train[['ups', 'author']].groupby(['author']).mean(), on='author', suffixes=(
    '', '_mean_author'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0'])

# Ajout mode ups author
test['Unnamed: 0'] = test.index

temp = train[['ups', 'author']].groupby(['author'])['ups'].agg(pd.Series.mode)
len_temp = len(temp)
for i in range(len_temp):
    if type(temp[i]) is np.ndarray:
        temp[i] = temp[i].mean()

test = test.merge(pd.DataFrame(temp), on='author', suffixes=(
    '', '_mode_author'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0'])

del temp, len_temp

# Ajout moyenne ups des réponse au commentaire père
test['Unnamed: 0'] = test.index
test = test.merge(train[['ups', 'parent_id']].groupby(['parent_id']).mean(), on='parent_id', suffixes=(
    '', '_mean_ans_parent'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0'])

# Ajout mode ups des réponse au commentaire père
test['Unnamed: 0'] = test.index
temp = train[['ups', 'parent_id']].groupby(
    ['parent_id'])['ups'].agg(pd.Series.mode)
for i in temp.index:
    if isinstance(temp[i], np.ndarray):
        temp[i] = temp[i].mean()

test = test.merge(pd.DataFrame(temp), on='parent_id', suffixes=(
    '', '_mode_ans_parent'), how='left').sort_values(['Unnamed: 0']).set_index(['Unnamed: 0'])

del temp


mean_author = train['ups_mean_author'].mean()
mean_author_parent = train['ups_mean_ans_parent'].mean()
#mod_author = train[['ups', 'author']].groupby(['author'])['ups'].agg(pd.Series.mode)
#mod_author_parent = train[['ups','parent_id']].groupby(['parent_id'])['ups'].agg(pd.Series.mode)

test['ups_mean_author'] = test['ups_mean_author'].fillna(mean_author)
test['ups_mode_author'] = test['ups_mode_author'].fillna(1)
test['ups_mean_ans_parent'] = test['ups_mean_ans_parent'].fillna(
    mean_author_parent)
test['ups_mode_ans_parent'] = test['ups_mode_ans_parent'].fillna(1)

# On fussionne les dataframes
df = pd.concat([train, test])

del train, test

# On ne garde que les features
df['ups_mode_author'] = df['ups_mode_author'].astype('float')
df['ups_mode_ans_parent'] = df['ups_mode_ans_parent'].astype('float')
df = df[['ups_mean_author', 'ups_mean_ans_parent',
         'ups_mode_author', 'ups_mode_ans_parent']]

# Export feature
df.to_csv(feature_dossier + "mean_mod_author.csv")

del df, mean_author, mean_author_parent

#### Creation de notre data frame totale

In [ ]:
# On prend les ups
data_all_feature = pd.read_csv(comments_students)[['ups']]

# Liste des chemins des features
liste_feat = [feature_dossier + "closeness.csv",
              feature_dossier + "nb_comm_per_author.csv",
              feature_dossier + "nb_mot_body.csv",
              feature_dossier + "pop_post.csv",
              feature_dossier + "sentiment_feature.csv",
              feature_dossier + "time_feature.csv",
              feature_dossier + "topic.csv",
              feature_dossier + "mean_mod_author.csv",
              feature_dossier + "post_comm.csv",
              feature_dossier + "closeness_author.csv",
              feature_dossier + "betweenness_centrality_author.csv"]

# On ajoute les dataframes un par un pour limiter l'impact sur la RAM
for i in tqdm(liste_feat):
    new_feat = pd.read_csv(i).drop(columns=['Unnamed: 0'])
    data_all_feature = pd.concat([data_all_feature, new_feat], axis=1)



100%|██████████| 11/11 [00:24<00:00,  2.23s/it]


In [ ]:
data_all_feature = pd.concat([data_all_feature, df["nombreDescendants","nombreEnfants","ratioEnfantsDescendants"]], axis=1)
# On export le csv avec toutes les features

data_all_feature.to_csv(feature_dossier + "data_all_feature.csv")

del data_all_feature, liste_feat, new_feat, i,df

# **PARTIE 2 :** Entraînement du modèle et prédictions




#### Chargement de notre data frame 

In [ ]:
# Chargement des features et separation entre train et test
df = pd.read_csv(feature_dossier + "data_all_feature.csv").drop(columns=['Unnamed: 0'])

### Séparation des données train et test

In [ ]:
## on a obtenu le score le plus optimale avec ces features
list_feat_T=['close_cent', 'link_new_id','Topic','bet_cent_aut','pop_post','time_answer','time_1st_comment_post','nb_com_aut','nombreDescendants','nombreEnfants','ratioEnfantsDescendants']

In [ ]:
# x_train/x_test
train = df[~np.isnan(df['ups'])]
x_train = train[list_feat_T]
y_train = pd.DataFrame(train['ups'])
test = df[np.isnan(df['ups'])]
x_test = test[list_feat_T]
del df,train,list_feat_T

### Entrainement du modele

In [ ]:
# entrainemenet du modele avec les parametres les plus optimaux trouvé
reg = GradientBoostingRegressor(loss='huber',criterion='friedman_mse',random_state=0,max_features='log2',n_estimators=500)
reg.fit(x_train,y_train)
  

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='huber',
                          max_depth=3, max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=0, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Prediction et extraction du csv finale

In [ ]:
y_test=reg.predict(x_test)
y_pred = pd.DataFrame(y_test)
y_pred.index = x_test.index
y_pred.columns = ['predicted']
# On récupère l'id des commentaires à prédire avec ups et on l'ajoute aux ups predites précédemment
df_fin = df_fin[np.isnan(df_fin['ups'])]
df_fin = df_fin.drop('ups', axis=1)
df_fin = pd.concat([df_fin, y_pred], axis=1)
df_fin.columns = ['id', 'predicted']

# Export du csv (penser à changer la version pour ne pas écraser un fichier csv précédent)

df_fin.to_csv("/content/drive/MyDrive/kaggle Project/Donnéespred_ups_5.csv", sep=',', index=False)

